# 2022 Bike Share Toronto Rides - Data Aggregation

### OBJECTIVE:

The purpose of this section of the project is to collect data from various sources to make a data frame that contains the following information:

| Column | Data Type |
| --- | --- |
| trip_id | int32 |  
| start_station_id | int16 |  
| end_station_id | int16 |  
| start_lat | float64 |
| start_lon | float64 |
| end_lat | float64 |
| end_lon | float64 |
| start_elevation | float64 |
| end_elevation | float64 |
| annual_member | bool |


### DATA SOURCES:
   * [Bike Share Ride Data - Toronto Open Data](https://open.toronto.ca/dataset/bike-share-toronto-ridership-data/)
   * [Bike Share Station Lat/Long - Toronto Open Data](https://open.toronto.ca/dataset/bike-share-toronto/)
   * [Elevation Data - Canada Open Government](https://open.canada.ca/data/en/dataset/f5c4e4af-ddc5-4f54-a70b-8390e4a4268e/resource/63ef74a5-4c9d-42b8-a0a7-68730a350759)
   
### STEPS:

   * [1. Get 2022 Ride Data](#1)
   * [2. Add Station Long/Lat](#2)
   * [3. Add Elevation Data](#3)
   * [4. Export Data](#4)

### 1. GET 2022 RIDE DATA <a class="anchor" id="1"></a>

In [1]:
# Import libraries
import glob
import json
import requests
import pandas as pd
import pyproj
import geopandas as gpd
import numpy as np
import rasterio as rio

In [2]:
# For each .csv file in the folder "data", read the specified columns and concatenate everything into one data frame
# Note: .csv files from Toronto Open Data contain an error double space in "Trip  Duration"
ride_cols = ['Trip Id', 'Start Station Id', 'End Station Id', 'Trip  Duration', 'Bike Id', 'User Type']
data = pd.concat(map(lambda x: pd.read_csv(x, usecols=ride_cols), glob.glob('data/*.csv')), ignore_index=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4620469 entries, 0 to 4620468
Data columns (total 6 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Trip Id           int64  
 1   Trip  Duration    int64  
 2   Start Station Id  int64  
 3   End Station Id    float64
 4   Bike Id           int64  
 5   User Type         object 
dtypes: float64(1), int64(4), object(1)
memory usage: 211.5+ MB


In [3]:
# Function to check data for 0 and null values
def checkNa0(df):
    return pd.concat(
            [pd.DataFrame(df.isna().sum(), columns=['is_na']),
            pd.DataFrame(df[(data == 0)].count(), columns=['is_0'])],
            axis=1
             )

checkNa0(data)

,is_na,is_0
Trip Id,0,0
Trip Duration,0,4591
Start Station Id,0,0
End Station Id,1279,0
Bike Id,0,0
User Type,0,0


In [4]:
# Drop NA "End Station Id" values
data = data.dropna()

# Drop rows where "Trip  Duration" is 0
data = data.loc[data['Trip  Duration'] != 0]

checkNa0(data)

,is_na,is_0
Trip Id,0,0
Trip Duration,0,0
Start Station Id,0,0
End Station Id,0,0
Bike Id,0,0
User Type,0,0


In [5]:
# Check for abnormalities in "User Type" (i.e. is there a category--such as "Employee"--that is unwanted?)
data['User Type'].unique()

array(['Casual Member', 'Annual Member'], dtype=object)

In [6]:
# Change 'User Type' into Boolean column
data['User Type'] = data['User Type'].map({'Casual Member': False, 'Annual Member': True})
data.rename(columns={'User Type': 'annual_member'}, inplace=True)

In [7]:
# Drop unneeded columns, which were only added to check for abnormalities in the data
data = data.drop(['Trip  Duration', 'Bike Id'], axis=1)

In [8]:
# Check max values in each column, to see what dtype each column can be cast down to
data.max()

Trip Id             20148783
Start Station Id        7760
End Station Id        7760.0
annual_member           True
dtype: object

In [9]:
# Change dtypes to save memory
for col in ['Start Station Id', 'End Station Id']:
    data[col] = data[col].astype('int16')

data['Trip Id'] = data['Trip Id'].astype('int32')

### 2. ADD STATION LONG/LAT <a class="anchor" id="2"></a>

In [10]:
# Request JSON bike station data, and put it into a data frame
json_request = requests.get('https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_information').content
station_data = pd.DataFrame(json.loads(json_request)['data']['stations'])[['station_id', 'lat', 'lon']].astype({
    'station_id': 'int',
})

# Add latitude and longitude of start station and end station to data frame, remove unneeded columns
data = pd.merge(data, station_data.rename(columns={'lat':'start_lat','lon':'start_lon'}),
                how='left', left_on='Start Station Id', right_on='station_id')
data = pd.merge(data, station_data.rename(columns={'lat':'end_lat','lon':'end_lon'}),
                how='left', left_on='End Station Id', right_on='station_id')
data = data.drop(columns=['station_id_x', 'station_id_y'])

In [11]:
# Check data for 0 and null values
checkNa0(data)

,is_na,is_0
Trip Id,0,0
Start Station Id,0,0
End Station Id,0,0
annual_member,0,2876857
start_lat,193154,0
start_lon,193154,0
end_lat,208227,0
end_lon,208227,0


In [12]:
# Drop NA lat/long values
data = data.dropna()

# Check data for 0 and null values
checkNa0(data)

,is_na,is_0
Trip Id,0,0
Start Station Id,0,0
End Station Id,0,0
annual_member,0,2656304
start_lat,0,0
start_lon,0,0
end_lat,0,0
end_lon,0,0


### 3. ADD ELEVATION DATA <a class="anchor" id="3"></a>

In [13]:
# Change previously used "station_data" data frame so it now holds all unique stations and their lat/long
#(to simplify retrieving elevation data)
station_data = pd.concat([
    data.loc[:,['Start Station Id', 'start_lat', 'start_lon']],
    data.loc[:,['End Station Id', 'end_lat', 'end_lon']].rename(columns={
    'End Station Id':'Start Station Id', 'end_lat':'start_lat', 'end_lon':'start_lon'
    })], ignore_index=True
)
station_data = station_data.rename(columns={'Start Station Id':'station_id', 'start_lat':'lat', 'start_lon':'lon'})
station_data = station_data.groupby('station_id').agg(func=max)
station_data = station_data.reset_index()

# Make a transformer to change from long/lat to utm. CRS codes were found on https://epsg.io/
lonlat_to_utm = pyproj.Transformer.from_crs(4326, 32617)

# Make a data frame of UTM coordinates for each stations
station_utm = pd.DataFrame(lonlat_to_utm.transform(station_data['lat'],station_data['lon'])).transpose()
station_utm.columns = ['utm_east', 'utm_north']


# Round UTM coordinates to the nearest 5, since that's the resolution of the elevation data
def roundTo5(x, base=5):
    return base * round(x/base)

station_utm = station_utm.apply(roundTo5)

# Update station_data and change dtypes
station_data = pd.concat([station_data, station_utm], axis=1)
station_data[['station_id','utm_east','utm_north']] = station_data[['station_id','utm_east','utm_north']].astype(int)

station_data.head()

,station_id,lat,lon,utm_east,utm_north
0,7000,43.639832,-79.395954,629380,4833120
1,7001,43.664964,-79.383550,630325,4835930
2,7002,43.667333,-79.399429,629040,4836170
3,7003,43.667158,-79.402761,628770,4836145
4,7004,43.656518,-79.389099,629895,4834985


In [14]:
# Make array of file locations of elevation data .tif files, originally downloaded from:
# https://open.canada.ca/data/en/dataset/f5c4e4af-ddc5-4f54-a70b-8390e4a4268e/resource/63ef74a5-4c9d-42b8-a0a7-68730a350759?inner_span=True
elevation_data = [
    './SW/dm_600481.tif', './SW/dm_600483.tif', './SW/dm_600485.tif',
    './SW/dm_620481.tif', './SW/dm_620483.tif', './SW/dm_620485.tif',
    './SW/dm_640483.tif', './SW/dm_640485.tif'
]

# Loop to change each filename in the array "elevation_data" to the content of that file as a data frame
for i, file in enumerate(elevation_data):

    # Import elevation data from .tif files to data frame
    with rio.Env():
        with rio.open(file) as src:

            # Create arrays for x and y axis of the raster using UTM EPSG:26917 values (coordinate system of source files)
            xmin, ymax = np.around(src.xy(0.00, 0.00), 9)
            xmax, ymin = np.around(src.xy(src.height-1, src.width-1), 9)
            x = np.linspace(xmin, xmax, src.width)
            y = np.linspace(ymax, ymin, src.height)  # max -> min so coords are top -> bottom

            # Create 2D arrays
            xs, ys = np.meshgrid(x, y) # I don't understand why this returns an array shape (4401,) ...shouldn't it be 2D?
            zs = src.read(1)

            # Apply NoData mask - i.e. only get rows for coordinates with elevation data
            mask = src.read_masks(1) > 0
            xs, ys, zs = xs[mask], ys[mask], zs[mask]

    df = {"utm_east": pd.Series(xs.ravel()),
            "utm_north": pd.Series(ys.ravel()),
            "elevation": pd.Series(zs.ravel())}

    elevation_data[i] = pd.DataFrame(data=df)

    # Remove elevation data for areas that have no bike stations, to save memory
    mask1 = elevation_data[i]['utm_east'] >= 617605
    mask2 = elevation_data[i]['utm_east'] <= 650980
    mask3 = elevation_data[i]['utm_north'] >= 4827155
    mask4 = elevation_data[i]['utm_north'] <= 4850075
    elevation_data[i] = elevation_data[i].loc[(mask1) & (mask2) & (mask3) & (mask4),:]

    # Change utm coordinates to int32, to save memory
    elevation_data[i]['utm_east'] = elevation_data[i]['utm_east'].astype('int32')
    elevation_data[i]['utm_north'] = elevation_data[i]['utm_north'].astype('int32')

# Combine elevation data into one data frame
elevation_data = pd.concat(elevation_data)

# Drop duplicate entries (I don't understand why there are duplicate entries)
elevation_data = elevation_data.drop_duplicates()

elevation_data.head()

,utm_east,utm_north,elevation
3521,617605,4830000,110.625053
3522,617610,4830000,110.591599
3523,617615,4830000,110.561623
3524,617620,4830000,110.524399
3525,617625,4830000,110.498726


In [15]:
# Add elevation to station_data
station_data = pd.merge(station_data, elevation_data, how='left', on=['utm_east', 'utm_north'])

station_data.head()

,station_id,lat,lon,utm_east,utm_north,elevation
0,7000,43.639832,-79.395954,629380,4833120,83.367996
1,7001,43.664964,-79.383550,630325,4835930,107.094704
2,7002,43.667333,-79.399429,629040,4836170,114.094002
3,7003,43.667158,-79.402761,628770,4836145,114.199348
4,7004,43.656518,-79.389099,629895,4834985,95.365051


In [16]:
# Check data for 0 and null values
checkNa0(station_data)

,is_na,is_0
station_id,0,0
lat,0,0
lon,0,0
utm_east,0,0
utm_north,0,0
elevation,1,0


In [17]:
# Find the row with the missing data
station_data[station_data['elevation'].isnull()]

,station_id,lat,lon,utm_east,utm_north,elevation
525,7623,43.60299,-79.492687,621650,4828885,NaN


In [18]:
# Since there is only one row with missing data, the missing or value will be added by looking at a close value:
elevation_data[(elevation_data['utm_east'] < 621650) & (elevation_data['utm_north'] == 4828885)
               ].sort_values(['utm_east', 'utm_north'], ascending=False)

,utm_east,utm_north,elevation
155363,620995,4828885,83.099304
155362,620990,4828885,83.094482
155361,620985,4828885,83.048294
155360,620980,4828885,83.000679
155359,620975,4828885,82.984505
...,...,...,...
895748,617625,4828885,102.391586
895747,617620,4828885,102.487068
895746,617615,4828885,102.525604
895745,617610,4828885,102.556213


In [19]:
# Add the value to station 7623
station_data.loc[station_data['station_id'] == 7623, 'elevation'] = 83.099304

# Check data for 0 and null values
checkNa0(station_data)

,is_na,is_0
station_id,0,0
lat,0,0
lon,0,0
utm_east,0,0
utm_north,0,0
elevation,0,0


In [20]:
# Add elevation to ride data
data = pd.merge(data, station_data[['station_id','elevation']].rename(columns={'elevation':'start_elevation'}),
                how='left', left_on='Start Station Id', right_on='station_id')
data = pd.merge(data, station_data[['station_id','elevation']].rename(columns={'elevation':'end_elevation'}),
                how='left', left_on='End Station Id', right_on='station_id')
data = data.drop(columns=['station_id_x', 'station_id_y'])

### 4. EXPORT DATA <a class="anchor" id="4"></a>

In [21]:
# Rename columns
data = data.rename(columns = {
    'Trip Id' : 'trip_id',
    'Start Station Id' : 'start_station_id',
    'End Station Id' : 'end_station_id'
})

# Export ride data
data.to_csv('bike_share_data.csv', index=False)